In [1]:
import pandas as pd
import numpy as np
# sklearn load_boston no longer works
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]

In [2]:
# make the data and target np arrays into a dataframe with appropriate column names
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']
data_values = pd.DataFrame(data, columns=column_names)
data_values['MEDV'] = target
data_values.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [3]:
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import StandardScaler

In [4]:
# declaring the features and target and scaling features
X = data_values[column_names]
y = data_values['MEDV']

scale = StandardScaler()

X = scale.fit_transform(X)

In [5]:
# creating testing and training splits
X_train, X_test, y_train, y_test = tts(X, y, test_size=0.2)

In [6]:
# helper function that will declare the model with the desired parameters, fit it, and print the mse
def get_mse(learning_rate, regularization):
  sgdr = SGDRegressor(eta0=learning_rate, penalty=regularization)
  sgdr.fit(X_train, y_train)
  y_pred = sgdr.predict(X_test)
  mse = mean_squared_error(y_true=y_test, y_pred=y_pred)
  print(f'Mean squared error (L{regularization[1:]} regularization & {learning_rate} learning rate):', mse)


In [7]:
# we want to test different learning rates (0.001, 0.01, 0.1) with both L1 and L2 regularization
l1 = 'l1'
# L1 regularization with eta = 0.001
get_mse(0.001, l1)
# L1 regularization with eta = 0.01
get_mse(0.01, l1)
# L1 regularization with eta = 0.1
get_mse(0.1, l1)

Mean squared error (L1 regularization & 0.001 learning rate): 29.728221011477512
Mean squared error (L1 regularization & 0.01 learning rate): 29.70260295088632
Mean squared error (L1 regularization & 0.1 learning rate): 36.825276940921206


In [8]:
# now do the above process with l2 regularization
l2 = 'l2'
# L2 regularization with eta = 0.001
get_mse(0.001, l2)
# L2 regularization with eta = 0.01
get_mse(0.01, l2)
# L2 regularization with eta = 0.1
get_mse(0.1, l2)

Mean squared error (L2 regularization & 0.001 learning rate): 29.720754345653738
Mean squared error (L2 regularization & 0.01 learning rate): 29.510925194330973
Mean squared error (L2 regularization & 0.1 learning rate): 31.20819687675843


In [9]:
# helper function that will fit each model 100 times and average the mse over those 100 times to get
# a better idea of which model performs better/worse on average
import statistics

def get_avg_mse(model):
  errors = []
  for i in range(100):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_true=y_test, y_pred=y_pred)
    errors.append(mse)
  return statistics.mean(errors)

In [10]:
# now each of those regularization/learning rate combos will be run 100 times and the errors of the
# trials will be averaged to determine more accurate error values
model = SGDRegressor(eta0=0.001, penalty='l1')
avg_mse = get_avg_mse(model)
print('The average mse after 100 trials for SGD regression with L1 regulzarization and a learning rate of 0.001:', avg_mse)
model = SGDRegressor(eta0=0.01, penalty='l1')
avg_mse = get_avg_mse(model)
print('The average mse after 100 trials for SGD regression with L1 regulzarization and a learning rate of 0.01:', avg_mse)
model = SGDRegressor(eta0=0.1, penalty='l1')
avg_mse = get_avg_mse(model)
print('The average mse after 100 trials for SGD regression with L1 regulzarization and a learning rate of 0.1:', avg_mse)
model = SGDRegressor(eta0=0.001, penalty='l2')
avg_mse = get_avg_mse(model)
print('The average mse after 100 trials for SGD regression with L2 regulzarization and a learning rate of 0.001:', avg_mse)
model = SGDRegressor(eta0=0.01, penalty='l2')
avg_mse = get_avg_mse(model)
print('The average mse after 100 trials for SGD regression with L2 regulzarization and a learning rate of 0.01:', avg_mse)
model = SGDRegressor(eta0=0.1, penalty='l2')
avg_mse = get_avg_mse(model)
print('The average mse after 100 trials for SGD regression with L2 regulzarization and a learning rate of 0.1:', avg_mse)

The average mse after 100 trials for SGD regression with L1 regulzarization and a learning rate of 0.001: 29.71953396749659
The average mse after 100 trials for SGD regression with L1 regulzarization and a learning rate of 0.01: 29.584702813214403
The average mse after 100 trials for SGD regression with L1 regulzarization and a learning rate of 0.1: 30.52072472173034
The average mse after 100 trials for SGD regression with L2 regulzarization and a learning rate of 0.001: 29.719923937469872
The average mse after 100 trials for SGD regression with L2 regulzarization and a learning rate of 0.01: 29.572993121427686
The average mse after 100 trials for SGD regression with L2 regulzarization and a learning rate of 0.1: 30.952393689622642


In [11]:
# now train AdaBoost Classifier on the data and compare the results
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

In [12]:
# declare decision stumps to use in AdaBoost
stump = DecisionTreeRegressor(max_depth=1)

In [13]:
# declare AdaBoost model
adaboost = AdaBoostRegressor(estimator=stump, n_estimators=5000, learning_rate=0.1)

In [14]:
# get predictions from AdaBoost model and get MSE to compare with linear regression models
adaboost.fit(X_train, y_train)
y_pred = adaboost.predict(X_test)

mse = mean_squared_error(y_true=y_test, y_pred=y_pred)
# using AdaBoost with strict decision stumps (max_depth = 1) performs poorly as the data has too
# many features and the stumps are not able to provide enough information
print('Mean squared error (AdaBoost Regression with decision stumps):', mse)

Mean squared error (AdaBoost Regression with decision stumps): 40.51757812866384


In [15]:
# get the average mse after 100 trials
print('The average mse after 100 trials for AdaBoost regression with decision stumps:', get_avg_mse(adaboost))

The average mse after 100 trials for AdaBoost regression with decision stumps: 41.261088556397596


In [16]:
# try AdaBoost with more complex base estimator
tree = DecisionTreeRegressor(max_depth=4)
adaboost = AdaBoostRegressor(estimator=tree, n_estimators=5000, learning_rate=0.1)

adaboost.fit(X_train, y_train)
y_pred = adaboost.predict(X_test)

mse = mean_squared_error(y_true=y_test, y_pred=y_pred)
# using AdaBoost with strict decision trees allow AdaBoost to gather more information
# and thus make better predictions
print('Mean squared error (AdaBoost Regression with decision trees):', mse)

Mean squared error (AdaBoost Regression with decision trees): 8.55864498197303


In [17]:
# get the average mse after 100 trials
print('The average mse after 100 trials for AdaBoost regression with decision trees:', get_avg_mse(adaboost))

The average mse after 100 trials for AdaBoost regression with decision trees: 8.521003392667275


In [18]:
# now try training a neural network to perform the regression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, InputLayer, BatchNormalization
from keras import regularizers

In [19]:
# define the neural network and add an input + hidden layers
nn = Sequential()
nn.add(InputLayer((X_train.shape[1],)))
# add a lay with regularization
nn.add(Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
# Batch normalization to normalize the outputs from each hidden layer before passing them along
# Batch normalization ensures that the data distribution remains consistent
nn.add(BatchNormalization())
nn.add(Dense(8, activation='relu'))
nn.add(BatchNormalization())
# linear activation function for output layer in regression
nn.add(Dense(1, activation='linear'))

In [20]:
# compile the neural network and fit it to the data
nn.compile(optimizer='adam', loss='mse', metrics=['mse'])

nn.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=8, verbose=1)

Epoch 1/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 578.0140 - mse: 576.5538 - val_loss: 492.7515 - val_mse: 491.2986
Epoch 2/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 532.5339 - mse: 531.0824 - val_loss: 489.7682 - val_mse: 488.3226
Epoch 3/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 590.7202 - mse: 589.2768 - val_loss: 482.1985 - val_mse: 480.7606
Epoch 4/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 584.5072 - mse: 583.0696 - val_loss: 474.2986 - val_mse: 472.8632
Epoch 5/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 517.1910 - mse: 515.7567 - val_loss: 463.7989 - val_mse: 462.3683
Epoch 6/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 508.9358 - mse: 507.5065 - val_loss: 456.2345 - val_mse: 454.8092
Epoch 7/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 497.9839 - mse: 496.5595 - val_loss: 446.0112 - val_mse: 444.5898
Epoch 8/100
41/41 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 497.4680 - mse: 496.0481 - val_loss: 432.5339 - val_mse: 431.1174


In [21]:
# now get predictions on the test data and mse
y_pred = nn.predict(X_test)

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


In [22]:
# it is clear that the neural network performed well but also required the most computational resources
# the neural network was able to capture nuances and nonlinearity in the data that the linear regression
# may not have been able to. However, AdaBoost with decision trees performed better and was both easier to
# write and computationally more efficient than a neural network
mse = mean_squared_error(y_pred=y_pred, y_true=y_test)
print('Mean squared error (neural network):', mse)

Mean squared error (neural network): 12.376206224918539


In [23]:
def get_avg_nn_mse():
  errors = []
  for i in range(100):
    # need to redefine neural network each time
    nn = Sequential()
    nn.add(InputLayer((X_train.shape[1],)))
    # add a lay with regularization
    nn.add(Dense(16, activation='relu', kernel_regularizer=regularizers.l2(0.1)))
    # Batch normalization to normalize the outputs from each hidden layer before passing them along
    # Batch normalization ensures that the data distribution remains consistent
    nn.add(BatchNormalization())
    nn.add(Dense(8, activation='relu'))
    nn.add(BatchNormalization())
    # linear activation function for output layer in regression
    nn.add(Dense(1, activation='linear'))
    # fit data and predict values

    nn.compile(optimizer='adam', loss='mse', metrics=['mse'])
    nn.fit(X_train, y_train, validation_split=0.2, epochs=100, batch_size=8, verbose=0)
    y_pred = nn.predict(X_test)
    mse = mean_squared_error(y_true=y_test, y_pred=y_pred)
    errors.append(mse)
    print(f'Trial {i+1} completed...')
  return statistics.mean(errors)

In [25]:
# get the average mse after 100 trials
print('The average mse after 100 trials for a neural network:', get_avg_nn_mse())

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Trial 1 completed...


1/4 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Trial 2 completed...
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
Trial 3 completed...
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Trial 4 completed...
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Trial 5 completed...
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Trial 6 completed...
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
Trial 7 completed...
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Trial 8 completed...
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Trial 9 completed...
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Trial 10 completed...
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Trial 11 completed...
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Trial 12 completed...
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Trial 13 completed...
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Trial 14 completed...
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Trial 15 completed...
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Trial 16 completed...
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
Trial 17 completed...
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Trial 18 c